# 用keras定义模型结构的两种方法

In [4]:
import tensorflow.keras as keras

In [11]:
# 方法一
model = keras.Sequential()
model.add(keras.layers.LSTM(2))
model.add(keras.layers.Dense(1))

In [13]:
# 方法二

layers = [keras.layers.LSTM(2), keras.layers.Dense(1)]
model = keras.Sequential(layers)

# 输入的数据

LSTM层要求的输入数据是三维的，分别代表样本数、时间步数、特征数  

# 第一层

第一层需要定义输入数据的形状  
例如：  

```python
model.add(LSTM(5, input_shape(2, 1)))
```

`input_shape`的参数代表输入数据的时间步数和特征数。  
默认的activation是linear  
5是什么意思？  

# 最后一层

回归问题：不需要特殊定义的最后一层  
二分类问题：`model.add(Activation('sigmoid'))`  
多分类问题：`model.add(Activation('softmax'))`  

# LSTM层

[?]  
默认情况下，LSTM的memory每个mini-batch会reset一次。  
设置参数`stateful=True`以后就不会自动reset。由开发者手动reset。    
以下情况会需要用到这一功能：  
1. batch_size = 1  
2. 一个长序列分成了几个子序列  
3. 一个非常长的序列所以要考虑效率问题  

## LSTM作为输入层

LSTM层要求的输入数据是三维的，分别代表样本数、时间步数、特征数。   
对应的，LSTM层也要通过参数`input_shape`定义可以接受的输入数据的形状。input_shape的参数代表输入数据的时间步数和特征数。  
默认的activation是linear。LSTM的第1个参数是输出向量的维度。   

In [4]:
import numpy as np

data = np.array([1,2,3,4,5,6,7,8,9,10])
data = data.reshape(1, 10, 1)
tf.keras.layers.LSTM(5, input_shape=(10, 1))

In [5]:
import numpy as np

data = np.array([[1,2],[3,4],[5,6],[7,8],[9,10]])
data = data.reshape(1, 5, 2)
tf.keras.layers.LSTM(5, input_shape=(5, 1))

## LSTM作为隐藏层

In [2]:
import tensorflow as tf

vocab_size = 10000
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

## 两个连续的LSTM层

In [10]:
import tensorflow as tf

vocab_size = 10000
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# 序列模型

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential(
    [
        # Embedding是指把编码转成指定维度的向量，使具有相近语义的单词其向量也是相近的
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        # Flatten也可以用GlobalAveragePooling1D代替，后者速度更快
        tf.keras.layers.Flatten().
        tf.keras.layers.Dense(6, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ]
)

# Embedding的可视化

In [ ]:
e = model.layers[0]   # embedding层
weights = e.get_weights()[0]
weights.shape()  # (10000, 16), 10000代表单词表的大小，16代表每个单词的维度
# 单词序号转换为单词
reverse_word_index = dict([(value, key) for (key, value) in word_index.items])

# meta.csv: 序号 --> 单词  
# vecs.csv：序号 --> embedding
import io
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size): # 0代表OOV
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + '\n')
    out_v.write('\t'.join(str(x) for x in embeddings)+'\n')
out_m.close()
out_v.close()
# https://projector.tensorflow.org

# 一维CNN做NLP

In [11]:
import tensorflow as tf

vocab_size = 10000
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# 语言生成模型

例如句子为：  
[1, 2, 3, 4, 5, 6, 7]  
[1, 3, 5, 7, 9]  
[2, 4, 6, 8]  
第一步：得到input_sequence为：  
[1, 2]  
[1, 2, 3]  
[1, 2, 3, 4]  
[1, 2, 3, 4, 5]  
[1, 2, 3, 4, 5, 6]  
[1, 2, 3, 4, 5, 6, 7]  
[1, 3]  
[1, 3, 5]  
[1, 3, 5, 7]  
[1, 3, 5, 7, 9]  
[2, 4]  
[2, 4, 6]  
[2, 4, 6, 8]  
第二步：以前面补0的方式全部对齐到同样的长度（长度为7）  
第三步：每一行的前6个为输入，最后一个为输出

In [ ]:
data = '.......'  # 训练数据
corpus = data.lower().split('\n')
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1 # +1代表OOV

input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequence.append(n_gram_sequence)
        
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences,
                                         maxlen = max_sequences_len,
                                         padding='pre'))
xs = input_sequences[:, :-1]
labels = input_sequences[:, -1]
ys = tf.keras.utils.to_categorical(labels, num_classes = totoal_words)

# 另一种写法

In [17]:
import tensorflow as tf

dataset = tf.data.Dataset.range(10)
for val in dataset:
    print (val.numpy())
dataset = dataset.window(5, shift=1, drop_remainder=True)
for window in dataset:
    print (window.numpy())
dataset = dataset.map(lambda window:window[:-1], window[-1:])
dataset = dataset.shuffle(buffer_size=10)
dataset = dataset.batch(2).prefetch(1)

0
1
2
3
4
5
6
7
8
9


AttributeError: '_VariantDataset' object has no attribute 'numpy'

# 分析lr

In [ ]:
import tensorflow as tf

l0 = tf.keras.layers.Dense(1, input_shape=[window_size])
model = tf.keras.models.Sequential([l0])
model.compile(loss='mse', optimizer=tf.keras.optimizers.SGD(lr=1e-6, momentum=0.9))
model.fit(dataset, epochs=100, verbose=0)
model.predict(series[1:21][np.newaxis])

lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch:1e-8 * 10**(epoch/20)
)
history = model.fit(dataset, epochs, callbacks=[lr_schedule])

得到下图的结果，选择底部平坦处中间位置的点作为真正的lr
![](assets/1.png)

# Seq2Seq模型

In [20]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    # 第二层RNN，如果没有`return_sequences=True`,只在最后一个时间步有输出。
    # 如果有`return_sequences=True`，则每个时间步有输出，就是seq2seq模型
    tf.keras.layers.SimpleRNN(20, return_sequences=True),
    tf.keras.layers.Dense(1)
])

# Lambda层

In [23]:
import tensorflow as tf

# 这句话放在keras的第一层，相当于数据的预处理
# input_shape=[None]代表输入为任意数据
tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1), input_shape=[None])

# 这句话放在最后一层，相当于输出数据的后期处理
tf.keras.layers.Lambda(lambda x: x*1000)

# CNN + LSTM

In [ ]:
import tensorflow as tf

models = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filter=32, kernel_size=5, strides=5, padding='causal', activation='relu', input_shape=['None', 1])
    tf.keras.layers.LSTM(32, return_sequences = True),
    tf.keras.layers.LSTM(32, return_sequences = True),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x:x*20)
])